In [2]:
from bs4 import BeautifulSoup
from time import sleep
import requests
from selenium import webdriver
import csv 
import pandas as pd 


def pmuScrapingRomande(url):
    ##------------------------ changes in this part only --------------------------------- ##
    driver = webdriver.Chrome()
    url = url
    driver.get(url)
    nameCsv = url[37:]
    # https://jeux.loro.ch/horses/raceCard/20200416_R2_R2C4?date=2020-04-16

    ### Pour une course déjà terminée
    # element = driver.find_element_by_xpath("""//*[@id="root"]/main/section/div/div/section/nav/ul/li[3]/p""")
    # element.click()

    ### supprimer des lettres spéciales
    def delSpeLetter(text):
        for x in text:
            if x not in 'abcdefghijklmnopqrstuvwyzABCDEFGHIJKLMNOPQRSTUVWYZ ':
                text = text.replace(x,'')
        return text

    ### on va regarder la liste des cheveaux qui vont faire de la course
    tempsAttente = 2

    sleep(tempsAttente)
    listePartante =[]
    listeNumber = driver.find_elements_by_class_name('participant-list__item-info__heading-number')
    sleep(tempsAttente)
    listeRun = driver.find_elements_by_class_name('participant-list__item-odds__number')
    sleep(tempsAttente)
    print('le nombre de chevaux dans la course')
    print(len(listeNumber))
    print('--'*20)
    print(len(listeRun))
    print('--'*20)
    for number,run in zip(listeNumber,listeRun):
        if run.text != '-':
            listePartante.append(number.text)
        sleep(tempsAttente)
    ### la liste des partants en int 

    listePartante = [int(x) for x in listePartante]
    print("la liste des chevaux partants")
    print(listePartante)

    ###chercher les informations des chevaux de la course
    links = driver.find_elements_by_class_name('collapsible__section')
    counterLinks = 0
    ## fiche de cheval
    listeInfos = [1,2,3,4,9,10]

    ### main programe

    ### écrire les données dans la un fichier csv
    global dataFile = 'data_{0}.csv'.format(nameCsv)
    with open(dataFile, mode='w') as horseRacing:
        horseRacing.write('number,name_horse,horse_age,weight,race_place,distance,prize,win_odd,draw,entraineur,proprietaire,sex\n')
        for link in links:
            if counterLinks < len(listePartante):

                ### le numéro de chaque cheval
                print('--'*20)
                print(listePartante[counterLinks])
                number = listePartante[counterLinks]
                print('--'*20)

                ### nom des chevaux
                horseName =driver.find_element_by_xpath("""
                //*[@id="root"]/main/section/div[1]/div/section/div[2]
                /div[1]/div/section[{0}]/header/div/div[1]/div/div/div
                /h2/span[2]
                """.format(listePartante[counterLinks]))
                print('Nom du cheval: ' + horseName.text)
                horse_name = delSpeLetter(horseName.text)
                ### la côte de chaque cheval
                odd = driver.find_element_by_xpath("""
                //*[@id="root"]/main/section/div/div/section/div/div[1]
                /div/section[{0}]/header/div/div[2]/span
                """.format(listePartante[counterLinks]))
                print('la côte: ' + odd.text)
                win_odd = odd.text
                ### cliquer sur les informations de chaque cheval
                link =driver.find_element_by_xpath("""
                //*[@id="root"]/main/section/div[1]/div/section/div[2]/div[1]/div/section[{0}]
                """.format(listePartante[counterLinks]))
            #     //*[@id="root"]/main/section/div/div/section/section/div[1]/div/section[{0}]
            #     //*[@id="root"]/main/section/div[1]/div/section/div[2]/div[1]/div/section[{0}]

                link.click()
                sleep(tempsAttente)
                ### fiche du cheval

                for x in listeInfos:
                    linke = driver.find_element_by_xpath("""
                    //*[@id="root"]/main/section/div/div/section/div[2]/div[1]/
                    div/section[{0}]/article/section/div/div/div[2]/div/div[2]/
                    div/table/tbody/tr[{1}]/td""".format(listePartante[counterLinks],x))

                    ## pour la course à l'avenir
                    """
                    //*[@id="root"]/main/section/div/div/section/div[2]/div[1]/
                    div/section[{0}]/article/section/div/div/div[2]/div/div[2]/
                    div/table/tbody/tr[{1}]/td
                    """

                    ## pour la course déjà terminé
                    """
                    //*[@id="root"]/main/section/div/div/section/section/div[1]
                    /div/section[{0}]/article/section/div/div/div[2]/div/div[2]
                    /div/table/tbody/tr[{1}]/td
                    """

                    if linke != None:
                        if x == 1 : 
                            print('Entraîneur: ' + linke.text)
                            entraineur = delSpeLetter(linke.text)
                        elif x == 2 :
                            print('Propriétaire: ' + linke.text)
                            proprietaire = delSpeLetter(linke.text)
                        elif x == 3 :
                            print('Sex: ' + linke.text)
                            sex = linke.text
                            if sex == 'MÂLE':
                                sex = 'MALE'
                        elif x == 4 :
                            print('Âge: ' + linke.text)
                            age = linke.text
                        elif x == 9 :
                            print('Poids(kg): ' + linke.text.replace('Kg',''))
                            poid = linke.text.replace('Kg','')
                        else:
                            print('Corde: ' + linke.text)
                            corde = linke.text

                    else:
                        print('hallo quoi !')
                    sleep(tempsAttente)

                ### changer de la fiche 
                sleep(tempsAttente)
                changeFiche = driver.find_element_by_xpath("""
                //*[@id="root"]/main/section/div[1]/div/section/div[2]
                /div[1]/div/section[{0}]/article/section/div/div/div[3]
                """.format(listePartante[counterLinks]))
                changeFiche.click()
                sleep(tempsAttente)

                ### performance 
                linkPlace = driver.find_element_by_xpath("""
                //*[@id="root"]/main/section/div[1]/div/section/div[2]
                /div[1]/div/section[{0}]/article/section/div/div/div[2]
                /div/div[3]/div/table[2]/tbody/tr[3]/td""".format(listePartante[counterLinks]))
                ### Places
                sleep(tempsAttente)
                print( 'Places: ' + linkPlace.text)
                place = linkPlace.text

                ### Pour la course déjà terminée
                """...
                """
                ### Pour la course à l’avenir
                """
                //*[@id="root"]/main/section/div/div/section/div[2]/div[1]
                /div/section[{0}]/article/section/div/div/div[2]/div/div[3]
                /div/table[2]/tbody/tr[3]/td
                """

                linkGain = driver.find_element_by_xpath("""
                //*[@id="root"]/main/section/div[1]/div/section/div[2]
                /div[1]/div/section[{0}]/article/section/div/div/div[2]
                /div/div[3]/div/table[2]/tbody/tr[4]/td""".format(listePartante[counterLinks]))
                sleep(tempsAttente)
                ### Gain en carrière
                print('Gain en carrière(frs): ' + linkGain.text.replace("'",""))
                gainCarriere = linkGain.text.replace("'","")

                ### xpath Pour la course déjà terminée
                """...
                """
                ### xpath Pour la course à l’avenir
                """
                //*[@id="root"]/main/section/div/div/section/div[2]/div[1]
                /div/section[{0}]/article/section/div/div/div[2]/div/div[3]
                /div/table[2]/tbody/tr[4]/td
                """

                ### Distance de la course 
                distance = driver.find_element_by_xpath("""
                //*[@id="root"]/main/div/article[2]/section/div/div[3]/div[1]/p
                """)
                print('Distance(m): ' + distance.text.replace('m',''))
                distanceCourse = distance.text.replace('m','')
                counterLinks += 1
                sleep(tempsAttente)
            else:
                break

            sleep(tempsAttente)
            horseRacing.write(str(number) + ','+ horse_name + ','+ age + ','+ poid + ','+ place + ','+ distanceCourse + ','+ gainCarriere + ','+ win_odd + ','+ corde + ','+ entraineur + ','+ proprietaire + ','+ sex + '\n')

    res = driver.execute_script("return document.documentElement.outerHTML")

    


In [3]:
pmuScrapingRomande('https://jeux.loro.ch/horses/raceCard/20200417_R3_R3C1')

le nombre de chevaux dans la course
10
----------------------------------------
10
----------------------------------------
la liste des chevaux partants
[2, 3, 4, 5, 6, 7, 8, 9]
----------------------------------------
2
----------------------------------------
Nom du cheval: FINAL STRYKE
la côte: 21
Entraîneur: Ms C Martindale
Propriétaire: M B Hutchinson, R J Mayo, B G St
Sex: HONGRE
Âge: 4
Poids(kg): 580 
Corde: 8
Places: 0
Gain en carrière(frs): 462
Distance(m): 1600
----------------------------------------
3
----------------------------------------
Nom du cheval: KWAHADI
la côte: 8.4
Entraîneur: R F Donat
Propriétaire: R F Donat Mrs B J Donat
Sex: HONGRE
Âge: 4
Poids(kg): 580 
Corde: 1
Places: 2
Gain en carrière(frs): 6524
Distance(m): 1600
----------------------------------------
4
----------------------------------------
Nom du cheval: SOCKAHOLIC
la côte: 19
Entraîneur: R Cluning
Propriétaire: R Cluning, Mrs E Cluning, Mr W M
Sex: HONGRE
Âge: 4
Poids(kg): 580 
Corde: 10
Places:

In [6]:
lire = pd.read_csv('data_20200417_R3_R3C1.csv')
lire.head(10)

,number,name_horse,horse_age,weight,race_place,distance,prize,win_odd,draw,entraineur,proprietaire,sex
0,2,FINAL STRYKE,4,580,0,1600,462,21.0,8,Ms C Martindale,M B Hutchinson R J Mayo B G St,HONGRE
1,3,KWAHADI,4,580,2,1600,6524,8.4,1,R F Donat,R F Donat Mrs B J Donat,HONGRE
2,4,SOCKAHOLIC,4,580,0,1600,380,19.0,10,R Cluning,R Cluning Mrs E Cluning Mr W M,HONGRE
3,5,STARCRIPPA,4,580,1,1600,2936,18.0,6,B M Cole,A J Prendergast M S Prendergast,HONGRE
4,6,STEEL SEDAKA,4,580,0,1600,1078,5.8,9,J P Salanitri,F Salanitri Archer A Puyat Mr,HONGRE
5,7,FINTONA QUEEN,4,560,0,1600,1157,16.0,7,K M Keys,Mrs L A Keys K M Keys R W Tayl,FEMELLE
6,8,ISOLA COMACINA,4,560,3,1600,8041,2.4,4,Andrew Noblet,R L Harding Mrs J E Harding,FEMELLE
7,9,KANSINO,4,560,1,1600,5819,3.9,2,C Maher D Eustace,Wylie Dalziel Roy Higgins Racing,FEMELLE


In [7]:
lire.columns

Index(['number', 'name_horse', 'horse_age', 'weight', 'race_place', 'distance',
       'prize', 'win_odd', 'draw', 'entraineur', 'proprietaire', 'sex'],
      dtype='object')